# Supplemental Code
- Inherent instability of simple DNA repeats shapes an evolutionarily stable distribution of repeat lengths 
- McGinty et al. 2025
- Part 3 of 4

In [ ]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import pickle
import os
# note: pandas interpolate function has a hidden dependency for scipy

import multiprocessing    # note: if using Windows, multiprocessing requires running Jupyter notebook via WSL

#### Load data

In [ ]:
# load data generated in Notebooks 1 and 2
# Can be skipped if running demo
CHM13_counts = pd.read_pickle('repeat_distributions/CHM13_counts.pickle')
CHM13_counts_B = pd.read_pickle('repeat_distributions/CHM13_counts_Bdist.pickle')
random_counts = pd.read_pickle('repeat_distributions/random_counts.pickle')
random_counts_B = pd.read_pickle('repeat_distributions/random_counts_Bdist.pickle')

denovo_exp_rate = pd.read_pickle('denovo/denovo_exp_rate_remake.pickle')
denovo_con_rate = pd.read_pickle('denovo/denovo_con_rate_remake.pickle')
denovo_nonexp_rate = pd.read_pickle('denovo/denovo_nonexp_rate_remake.pickle')

denovo_exp_rate_poisson = pd.read_pickle('denovo/denovo_exp_rate_poisson_remake.pickle')
denovo_con_rate_poisson = pd.read_pickle('denovo/denovo_con_rate_poisson_remake.pickle')
denovo_nonexp_rate_poisson = pd.read_pickle('denovo/denovo_nonexp_rate_poisson_remake.pickle')

denovo_substitution_context_rate = pd.read_pickle('denovo/denovo_mut_freq_all.pickle')
denovo_substitution_context_rate_poisson = pd.read_pickle('denovo/denovo_mut_freq_all_poisson.pickle')

rates_mu_nu = pd.read_pickle('denovo/denovo_mut_freq_mu_nu_remake.pickle')

intercept_list = dict()
intercept_list['A'] = [denovo_con_rate['A'][8]] + [denovo_exp_rate['A'][8] * (denovo_exp_rate['A'][8]/ denovo_con_rate['A'][8])**x for x in range(7)]

##### skip this section if file has not yet been generated (see below in Notebook)

In [ ]:
subonly_counts = pd.read_pickle('repeat_distributions/subonly_counts_remake.pickle')

In [ ]:
# uniform distribution for L=6-70
counts_uniformrange = subonly_counts.copy().T
for i in range(6,71):
    counts_uniformrange[i] = counts_uniformrange[10].copy()
counts_uniformrange = counts_uniformrange.T
counts_uniformrange['A'] = counts_uniformrange['A'] * (subonly_counts.sum()['A'] / counts_uniformrange.sum()['A'])
counts_uniformrange['B'] = subonly_counts['B'].copy()
counts_uniformrange = counts_uniformrange.copy()
counts_uniformrange.to_pickle('repeat_distributions/subonly_counts_uniformrange.pickle')

In [ ]:
counts_uniform = pd.read_pickle('repeat_distributions/subonly_counts_uniformrange.pickle')

#### Define functions

In [ ]:
# evolve function
def mut_evolve_dist_AB(A_count_input, B_count_input, starting_conditions, boot = None, input_nuc = 'A', mut = True, mutonly = False, speedup_multiplier = 1, output_components = False, stochastics = None, reflective = True, boundary_count = 1000, first_bin = 0):
    exp_rate_A_AA, con_rate_A_AA, nonexp_rate_A_AB, B_indel_rates = starting_conditions
    A_count_output = A_count_input.copy(); B_count_output = B_count_input.copy()
    A_bins = len(A_count_input); B_bins = len(B_count_input)
    boundary_flag = False
    A_length_array = np.array(range(1,A_bins+3))
    A_length_array_bases = np.array(range(1,A_bins+3)) * len(input_nuc) ### including motif length
    B_length_array = np.array(range(1,B_bins+3))
    B_length_array_bases = np.array(range(1,B_bins+3)) * len(input_nuc) ### including motif length
    A_count_input = np.insert(A_count_input, A_bins, [0,0])
    B_count_input = np.insert(B_count_input, B_bins, [0,0])

    if boot is None:
        denovo_sub = denovo_substitution_context_rate.loc[input_nuc]
    else:
        denovo_sub = denovo_substitution_context_rate_poisson.loc[boot].loc[input_nuc]

    # distribution info
    total_B_bases = (B_count_input[:B_bins] * B_length_array_bases[:B_bins]).sum()
    B_L1_base_portion = ((B_count_input[0] * len(input_nuc)) / (B_count_input[:B_bins]* B_length_array_bases[:B_bins]).sum()) ### including motif length
    B_nonflank_base_portion = (B_count_input[2:B_bins+2] * B_length_array_bases[:B_bins]).sum() / total_B_bases  ### include portion of triplets 1nt away???
    B_flank_base_portion = (B_count_input[1:B_bins] * 2 * len(input_nuc)).sum() / total_B_bases ### including motif length
    
    total_A_bases = (A_count_input[:A_bins] * A_length_array_bases[:A_bins]).sum()
    A_nonflank_base_portion = (A_count_input[2:A_bins+2] * A_length_array_bases[:A_bins]).sum() / total_A_bases
    A_flank_base_portion = (A_count_input[1:A_bins] * 2 * len(input_nuc)).sum() / total_A_bases ### including motif length
    
    total_A_change_in = np.array([0.0]*A_bins); total_B_change_in = np.array([0.0]*B_bins)
    total_A_change_out = np.array([0.0]*A_bins); total_B_change_out = np.array([0.0]*B_bins)

    if mut == True:
        # A>B which adds to the A count locally. add these to A
        A_mut_in_local_A_B = 2 * len(input_nuc) * denovo_sub['Acontraction'] * A_count_input[1:]
        A_mut_out_local_A_B = -2 * len(input_nuc) * denovo_sub['Acontraction'] * A_count_input
        A_mut_out_local_A_B[0] = -1 * len(input_nuc) * A_count_input[0] * denovo_sub['A10']

        # total number of A>B fission events
        A_mut_out_fission = np.insert((-denovo_sub['Afission'] * A_count_input[2:] * A_length_array_bases[:A_bins]), 0, [0, 0]) # used to subtract from A_count, starting from L=3 (with 0 for L=1,2)
        # each fission creates 2 As. add these to A
        A_mut_in_fission = ((2/A_length_array[:A_bins]) * -A_mut_out_fission[2:A_bins+2])[::-1].cumsum()[::-1]
 
        # B>A which adds to the A count locally (which must come from B_L>1)
        # A from B>A leaving the -1 bin
        A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
        A_mut_out_local_B_A = -denovo_sub['Aexpansion'] * B_flank_base_portion * total_B_bases * A_len_freq
        # B>A creating A_L=1 from B_L>2
        B_A_into_L1 = total_B_bases * B_nonflank_base_portion * denovo_sub['A01']
        A_mut_in_local_B_A = np.insert(-A_mut_out_local_B_A, 0, B_A_into_L1)
        
        # fusion process for A
        A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
        A_fusion_freq_in = np.bincount((np.add.outer(A_length_array[:A_bins], A_length_array[:A_bins])+1).ravel(), weights = np.outer(A_len_freq, A_len_freq).ravel())[1:]
        A_mut_in_fusion_A_B = A_fusion_freq_in * denovo_sub['Afusion'] * B_L1_base_portion * total_B_bases
        A_mut_out_fusion_A_B = (-2) *A_len_freq * denovo_sub['Afusion'] * B_L1_base_portion * total_B_bases
        
        # total B>A
        # B>A which adds to the B count locally. add these to B
        B_mut_in_local_B_A = 2 * len(input_nuc) * denovo_sub['Aexpansion'] * B_count_input[1:]
        B_mut_out_local_B_A = -2 * len(input_nuc) * denovo_sub['Aexpansion'] * B_count_input
        B_mut_out_local_B_A[0] = -1 * B_L1_base_portion * total_B_bases * denovo_sub['Afusion']

        # total number of B>A fission events
        B_mut_out_fission = np.insert((-denovo_sub['A01'] * B_count_input[2:] * B_length_array_bases[:B_bins]), 0, [0, 0]) # used to subtract from B_count, starting from L=3 (with 0 for L=1,2)
        # each fission creates 2 Bs. add these to B
        B_mut_in_fission = ((2/B_length_array[:B_bins]) * -B_mut_out_fission[2:B_bins+2])[::-1].cumsum()[::-1]

        # A>B which adds to the B count locally (which must come from A_L>1)
        # B from A>B leaving the -1 bin
        B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
        B_mut_out_local_A_B = -denovo_sub['Acontraction'] * A_flank_base_portion * total_A_bases * B_len_freq
        # A>B creating B_L=1 from A_L>2
        A_B_into_L1 = total_A_bases * A_nonflank_base_portion * denovo_sub['Afission']
        B_mut_in_local_A_B = np.insert(-B_mut_out_local_A_B, 0, A_B_into_L1)
        
        # fusion process for B
        B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
        B_fusion_freq_in = np.bincount((np.add.outer(B_length_array[:B_bins], B_length_array[:B_bins])+1).ravel(), weights = np.outer(B_len_freq, B_len_freq).ravel())[1:]
        B_mut_in_fusion_B_A = B_fusion_freq_in * denovo_sub['A10'] * A_count_input[0] * len(input_nuc)
        B_mut_out_fusion_B_A = (-2) * B_len_freq * denovo_sub['A10'] * A_count_input[0] * len(input_nuc)

        # update counts for next round (with absorbing boundary)
        total_A_change_in += A_mut_in_local_A_B[:A_bins] + A_mut_in_local_B_A[:A_bins] + A_mut_in_fission[:A_bins] + A_mut_in_fusion_A_B[:A_bins]
        total_B_change_in += B_mut_in_local_B_A[:B_bins] + B_mut_in_local_A_B[:B_bins] + B_mut_in_fission[:B_bins] + B_mut_in_fusion_B_A[:B_bins]
        total_A_change_out += A_mut_out_local_A_B[:A_bins] + A_mut_out_local_B_A[:A_bins] + A_mut_out_fission[:A_bins] + A_mut_out_fusion_A_B[:A_bins]
        total_B_change_out += B_mut_out_local_B_A[:B_bins] + B_mut_out_local_A_B[:B_bins] + B_mut_out_fission[:B_bins] + B_mut_out_fusion_B_A[:B_bins]

        # apply reflecting boundary
        if reflective == True:
            total_A_change_in[A_bins-1] += A_mut_in_local_A_B[A_bins:].sum() + A_mut_in_local_B_A[A_bins:].sum() + A_mut_in_fission[A_bins:].sum() + A_mut_in_fusion_A_B[A_bins:].sum()
            total_B_change_in[B_bins-1] += B_mut_in_local_B_A[B_bins:].sum() + B_mut_in_local_A_B[B_bins:].sum() + B_mut_in_fission[B_bins:].sum() + B_mut_in_fusion_B_A[B_bins:].sum()
            total_A_change_out[A_bins-1] += A_mut_out_local_A_B[A_bins:].sum() + A_mut_out_local_B_A[A_bins:].sum() + A_mut_out_fission[A_bins:].sum() + A_mut_out_fusion_A_B[A_bins:].sum()
            total_B_change_out[B_bins-1] += B_mut_out_local_B_A[B_bins:].sum() + B_mut_out_local_A_B[B_bins:].sum() + B_mut_out_fission[B_bins:].sum() + B_mut_out_fusion_B_A[B_bins:].sum()
           
    if mutonly == False:
        # A expansions in and out
        A_exp_out = A_count_input[:A_bins] * -exp_rate_A_AA[:A_bins]
        A_exp_in = np.insert(-A_exp_out, 0, B_indel_rates[0]*total_B_bases)

        # A contractions in and out
        A_con_out = A_count_input[:A_bins+1] * -con_rate_A_AA[:A_bins+2]
        A_con_in = -A_con_out[1:]

        # A fusions from B1>B0 deletions
        if (mut != True):
            A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
            A_fusion_freq_in = np.bincount((np.add.outer(A_length_array[:A_bins], A_length_array[:A_bins])+1).ravel(), weights = np.outer(A_len_freq, A_len_freq).ravel())[1:]
        A_mut_in_fusion_Bdel = A_fusion_freq_in[1:A_bins+1] * B_indel_rates[1] * B_L1_base_portion * total_B_bases
        A_mut_out_fusion_Bdel = (-2) *A_len_freq * B_indel_rates[1] * B_L1_base_portion * total_B_bases

        # A fission events from insertions
        A_nonexp_out_fission = -A_count_input * nonexp_rate_A_AB # used to calculate fission_in, starting with L=2 going to 2x L=1
        # each fission creates 2 As. add these to A
        A_nonexp_in_fission = ((2/A_length_array[:A_bins]) * -A_nonexp_out_fission[1:A_bins+1])[::-1].cumsum()[::-1]

        # B expansions in and out
        B_exp_out = B_count_input[:B_bins] * -B_indel_rates[2] * B_length_array[:B_bins] # B>BB rates are flat, per base
        B_exp_in = np.insert(-B_exp_out, 0, A_nonexp_out_fission.sum())

        # B contractions in and out
        B_con_out = B_count_input[:B_bins+1] * -B_indel_rates[1] * B_length_array[:B_bins+1] # B>_ rates are flat, per base
        B_con_in = -B_con_out[1:]

        # B fusions from A1>A0 deletions
        if (mut != True):
            B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
            B_fusion_freq_in = np.bincount((np.add.outer(B_length_array[:B_bins], B_length_array[:B_bins])+1).ravel(), weights = np.outer(B_len_freq, B_len_freq).ravel())[1:]
        B_mut_in_fusion_Adel = B_fusion_freq_in[1:B_bins+1] * A_count_input[0] * con_rate_A_AA[0]
        B_mut_out_fusion_Adel = 2 *B_len_freq * A_count_input[0] * -con_rate_A_AA[0]
                    
        # B fission events from insertions
        B_nonexp_out_fission = -B_count_input * B_indel_rates[0] * B_length_array # used to calculate fission_in, starting with L=2 going to 2x L=1
        # each fission creates 2 Bs. add these to B
        B_nonexp_in_fission = ((2/B_length_array[:B_bins]) * -B_nonexp_out_fission[1:B_bins+1])[::-1].cumsum()[::-1]

       # update counts for next round (with absorbing boundary)
        total_A_change_in += A_exp_in[:A_bins] + A_con_in[:A_bins] + A_mut_in_fusion_Bdel[:A_bins] + A_nonexp_in_fission[:A_bins]
        total_B_change_in += B_exp_in[:B_bins] + B_con_in[:B_bins] + B_mut_in_fusion_Adel[:B_bins] + B_nonexp_in_fission[:B_bins]
        total_A_change_out += A_exp_out[:A_bins] + A_con_out[:A_bins] + A_mut_out_fusion_Bdel[:A_bins] + A_nonexp_out_fission[:A_bins]
        total_B_change_out += B_exp_out[:B_bins] + B_con_out[:B_bins] + B_mut_out_fusion_Adel[:B_bins] + B_nonexp_out_fission[:B_bins]

        # apply reflecting boundary
        if reflective == True:
            total_A_change_in[A_bins-1] += A_exp_in[A_bins:].sum() + A_con_in[A_bins:].sum() + A_mut_in_fusion_Bdel[A_bins:].sum() + A_nonexp_in_fission[A_bins:].sum()
            total_B_change_in[B_bins-1] += B_exp_in[B_bins:].sum() + B_con_in[B_bins:].sum() + B_mut_in_fusion_Adel[B_bins:].sum() + B_nonexp_in_fission[B_bins:].sum()
            total_A_change_out[A_bins-1] += A_exp_out[A_bins:].sum() + A_con_out[A_bins:].sum() + A_mut_out_fusion_Bdel[A_bins:].sum() + A_nonexp_out_fission[A_bins:].sum()
            total_B_change_out[B_bins-1] += B_exp_out[B_bins:].sum() + B_con_out[B_bins:].sum() + B_mut_out_fusion_Adel[B_bins:].sum() + B_nonexp_out_fission[B_bins:].sum()

    # flag to stop the simulation if more repeats are removed from a bin than exist in that bin (excluding the last 10 noisy bins)
    flag = ((np.abs(total_A_change_out[:A_bins-10]) * speedup_multiplier > A_count_output[:A_bins-10]).sum()) > 0

    # apply speedup
    total_A_change_in *= speedup_multiplier; total_A_change_out *= speedup_multiplier
    total_B_change_in *= speedup_multiplier; total_B_change_out *= speedup_multiplier
    
    if stochastics is not None:
        # the sum of poisson random variables is poisson-distributed. not necessary to run n poisson samples
        total_A_change_in = np.random.poisson(total_A_change_in.clip(0))
        total_A_change_out = -1 * np.random.poisson(np.abs(total_A_change_out.clip(max=0)))
        total_B_change_in = np.random.poisson(total_B_change_in.clip(0))
        total_B_change_out = -1 * np.random.poisson(np.abs(total_B_change_out.clip(max=0)))   
    
    total_A_change = total_A_change_in + total_A_change_out
    total_B_change = total_B_change_in + total_B_change_out

    if first_bin != 0:
        total_A_change[:first_bin] = 0; total_B_change[:first_bin] = 0
        
    # update counts for next round
    A_count_output = A_count_output[:A_bins] + total_A_change[:A_bins]
    B_count_output = B_count_output[:B_bins] + total_B_change[:B_bins]

    # remove negative values
    A_count_output[A_count_output <0] = 0            
    B_count_output[B_count_output <0] = 0

    boundary_flag = ((A_count_output[A_bins-1] > boundary_count) | (B_count_output[B_bins-1] > boundary_count))
    
    if output_components == True:
        if mutonly == False:
            return  A_mut_in_local_A_B[:A_bins], A_mut_out_local_A_B[:A_bins], A_mut_in_local_B_A[:A_bins], A_mut_out_local_B_A[:A_bins], A_mut_in_fission[:A_bins], A_mut_out_fission[:A_bins], A_mut_in_fusion_A_B[:A_bins], A_mut_out_fusion_A_B[:A_bins], A_exp_in[:A_bins], A_exp_out[:A_bins], A_con_in[:A_bins], A_con_out[:A_bins], A_mut_in_fusion_Bdel[:A_bins], A_mut_out_fusion_Bdel[:A_bins], A_nonexp_in_fission[:A_bins], A_nonexp_out_fission[:A_bins]
        else:
            return  A_mut_in_local_A_B[:A_bins], A_mut_out_local_A_B[:A_bins], A_mut_in_local_B_A[:A_bins], A_mut_out_local_B_A[:A_bins], A_mut_in_fission[:A_bins], A_mut_out_fission[:A_bins], A_mut_in_fusion_A_B[:A_bins], A_mut_out_fusion_A_B[:A_bins]
    else:
        return A_count_output, B_count_output, flag, boundary_flag

In [ ]:
def pin_power_law(power, pin_rate, pin_len=9, start_len=1, end_len=200):
    denom = (pin_len**power) / pin_rate
    return pd.Series([i**power for i in range(start_len, end_len+1)], index = list(range(start_len,end_len+1))) / denom

def intercept_then_powerlaw(exp_power, con_power, exp_int, con_int, pin_len = 9, A_bins = 200, boot = None, motif = 'A', interp = False, nonexp_factor = 0.01, intercept_group = 'motif'):
    if intercept_group == 'motif':
        intercept_group = intercept_list[motif]
    if boot is None:
        bootname = ''
        denovo_exp_rate_current = denovo_exp_rate[motif].copy()
        denovo_con_rate_current = denovo_con_rate[motif].copy()
        denovo_nonexp_rate_current = denovo_nonexp_rate[motif].copy()
    else:
        bootname = '_boot'+str(boot)
        denovo_exp_rate_current = denovo_exp_rate_poisson[motif][boot].copy()
        denovo_con_rate_current = denovo_con_rate_poisson[motif][boot].copy()
        denovo_nonexp_rate_current = denovo_nonexp_rate_poisson[motif][boot].copy()
    exp = pd.concat([denovo_exp_rate_current.reindex(range(pin_len)), pin_power_law(exp_power, intercept_group[exp_int], start_len=pin_len, end_len=A_bins+3)])
    con = pd.concat([denovo_con_rate_current.reindex(range(pin_len)), pin_power_law(con_power, intercept_group[con_int], start_len=pin_len, end_len=A_bins+3)])
    nonexp = pd.concat([denovo_nonexp_rate_current.reindex(range(pin_len)), pin_power_law(exp_power, intercept_group[exp_int] * nonexp_factor, start_len=pin_len, end_len=A_bins+3)])
    nonexpname = '_nex' + str(nonexp_factor)
    nonexp.loc[1] = 0
    if interp == True:
        interpname = '_interp'
        exp.loc[8:13] = np.nan
        con.loc[8:13] = np.nan
        nonexp.loc[8:13] = np.nan
        exp = exp.interpolate(method = 'quadratic')
        con = con.interpolate(method = 'quadratic')
        nonexp = nonexp.interpolate(method = 'quadratic')
    else:
        interpname = ''
    name = '_interceptPL_tE' + str(exp_power) + '_tC' + str(con_power) +'_iE' + str(exp_int) + '_iC' + str(con_int) + nonexpname + interpname + bootname
    return name, exp, con, nonexp

#### alternate rate functional forms
def power_law_rate_at_L(power, L, rate = 1e-8, bins = 200):
    return pd.Series([np.exp(np.log(rate) - power * np.log(L) + np.log(length)*power) for length in range(1,bins)], index = range(1,bins))

def power_law_rate_at_L_v2(power, Lam, mu = 4.117e-09, bins = 200):
    return pd.Series([((2*mu)/Lam) * (length/Lam)**power for length in range(1,bins)], index = range(1,bins))

def log_curve_from_L9(power, intercept, motif = 'A', bins = 200):
    return pd.Series([intercept_list[motif][intercept] * (np.log(1+L-8) / np.log(1+9 -8))**power for L in range(9,bins+1)], index = range(9,bins+1))

In [ ]:
# setup function
def setup_evolve(exp_power=0, con_power=0, exp_int=0, con_int=0, boot = None, stochastics = None, interp = False, pin_len = 9, nonexp_factor = 0.01, A_bins = 200, B_bins = 200, input_nuc = 'A', mutonly = False, exp_zero = False, con_zero = False, nonexp_zero = False, starting_counts = 'random', ceiling = None, rates_function = 'powerlaw'):
# set up counts
    A_length_array = np.array(range(1,A_bins+1))
    B_length_array = np.array(range(1,A_bins+1))
    if starting_counts == 'random':
        A_count_input = np.nan_to_num(random_counts[len(input_nuc)][input_nuc].reindex(range(1,A_bins+1)).values)
        B_count_input = np.nan_to_num(random_counts_B[input_nuc].reindex(range(1,B_bins+1)).values)
    if starting_counts == 'subonly':
        A_count_input = np.nan_to_num(subonly_counts['A'][input_nuc].reindex(range(1,A_bins+1)).values)
        B_count_input = np.nan_to_num(subonly_counts['B'][input_nuc].reindex(range(1,B_bins+1)).values)
    if starting_counts == 'CHM13':
        A_count_input = np.nan_to_num(CHM13_counts[len(input_nuc)][input_nuc].reindex(range(1,A_bins+1)).values)
        B_count_input = np.nan_to_num(CHM13_counts_B[input_nuc].reindex(range(1,B_bins+1)).values)
    if starting_counts == 'uniform':
        A_count_input = np.nan_to_num(counts_uniform['A'][input_nuc].reindex(range(1,A_bins+1)).values)
        B_count_input = np.nan_to_num(counts_uniform['B'][input_nuc].reindex(range(1,B_bins+1)).values)
    if starting_counts not in ['random', 'subonly', 'CHM13', 'uniform']:
        A_count_input = np.nan_to_num(starting_counts[0].reindex(range(1,A_bins+1)).values)
        B_count_input = np.nan_to_num(starting_counts[1].reindex(range(1,B_bins+1)).values)
# set up rates    
    if mutonly == False:
        if rates_function == 'powerlaw':
            name, exp_rate, con_rate, nonexp_rate = intercept_then_powerlaw(exp_power = exp_power, con_power = con_power, exp_int = exp_int, con_int=con_int, pin_len=pin_len, A_bins = A_bins, boot = boot, motif = input_nuc, interp = interp, nonexp_factor = nonexp_factor)
            B_indel_rate = np.array([exp_rate[0], con_rate[0], nonexp_rate[0]])        
        if rates_function == 'powerlaw_x':
            name = '_PL1e-8atL_tE' + str(exp_power) + '_tC' + str(con_power) +'_LE' + str(exp_int) + '_LC' + str(con_int)
            exp_rate = power_law_rate_at_L(exp_power, exp_int, rate = rates_mu_nu['B>A'], bins = A_bins+3).reindex(range(A_bins+3))
            con_rate = power_law_rate_at_L(con_power, con_int, rate = rates_mu_nu['A>B'], bins = A_bins+3).reindex(range(A_bins+3))
            nonexp_rate = exp_rate * nonexp_factor
            B_indel_rate = np.array([denovo_exp_rate[input_nuc][0], denovo_con_rate[input_nuc][0], denovo_nonexp_rate[input_nuc][0]])
        if rates_function == 'powerlaw_lambda':
            name = '_PLmuatL_tE' + str(exp_power) + '_tC' + str(con_power) +'_LE' + str(exp_int) + '_LC' + str(con_int)
            exp_rate = power_law_rate_at_L_v2(exp_power, exp_int, mu = rates_mu_nu['B>A'], bins = A_bins+3).reindex(range(A_bins+3))
            con_rate = power_law_rate_at_L_v2(con_power, con_int, mu = rates_mu_nu['B>A'], bins = A_bins+3).reindex(range(A_bins+3))
            nonexp_rate = exp_rate * nonexp_factor
            B_indel_rate = np.array([denovo_exp_rate[input_nuc][0], denovo_con_rate[input_nuc][0], denovo_nonexp_rate[input_nuc][0]])
        if rates_function == 'log':
            name = '_logrates_tE' + str(exp_power) + '_tC' + str(con_power) +'_iE' + str(exp_int) + '_iC' + str(con_int)
            exp_rate = pd.concat([denovo_exp_rate['A'].loc[:8], log_curve_from_L9(exp_power, exp_int, A_bins+3)])
            con_rate = pd.concat([denovo_con_rate['A'].loc[:8], log_curve_from_L9(con_power, con_int, A_bins+3)])
            nonexp_rate = pd.concat([denovo_nonexp_rate['A'].loc[:8], log_curve_from_L9(exp_power, exp_int, A_bins+3) * nonexp_factor])
            B_indel_rate = np.array([denovo_exp_rate[input_nuc][0], denovo_con_rate[input_nuc][0], denovo_nonexp_rate[input_nuc][0]])
        if rates_function == 'custom':
            exp_rate, con_rate, name = custom_rates
            exp_rate = exp_rate.reindex(range(A_bins+3))
            con_rate = con_rate.reindex(range(A_bins+3))
            nonexp_rate = exp_rate * nonexp_factor
            B_indel_rate = np.array([denovo_exp_rate[input_nuc][0], denovo_con_rate[input_nuc][0], denovo_nonexp_rate[input_nuc][0]])
        # change rates from per unit to per STR
        exp_rate = exp_rate.values[1:A_bins+1] * A_length_array
        con_rate = con_rate.values[1:A_bins+2] * np.array(range(1,A_bins+2))
        nonexp_rate = nonexp_rate.values[1:A_bins+3] * np.array(range(1,A_bins+3))
        if ceiling != None:
            ceiling_loc = []
            if (exp_rate > ceiling).sum() > 0:
                ceiling_loc.append(pd.Series(exp_rate > ceiling).idxmax())
            if (con_rate > ceiling).sum() > 0:
                ceiling_loc.append(pd.Series(con_rate > ceiling).idxmax())
            if len(ceiling_loc) > 0:
                ceiling_loc = min(np.array(ceiling_loc))
                print( '\r' + 'rate ceiling reached at L=' + str(ceiling_loc), end = ' ')
                exp_rate[ceiling_loc:] = ceiling
                con_rate[ceiling_loc:] = ceiling
                nonexp_rate[ceiling_loc:] = ceiling
            name = name + '_ceiling_' + str(ceiling)
        if exp_zero == True:
            exp_rate *= 0
        if con_zero == True:
            con_rate *= 0
        if nonexp_zero == True:
            nonexp_rate *= 0
        if starting_counts == 'random':
            name = name + '_randomstart'        
        if starting_counts == 'subonly':
            name = name + '_subonlystart'
        if starting_counts == 'CHM13':
            name = name + '_CHM13start'
        if starting_counts == 'uniform':
            name = name + '_uniformstart'
        if stochastics is None:
            name = name
        else:
            name = name + '_stochastics_' + str(stochastics)
        return A_count_input, B_count_input, exp_rate, con_rate, nonexp_rate, B_indel_rate, name
    else:
        if boot is None:
            name = 'mutonly'
        else:
            name = 'mutonly_boot' + str(boot)
        if starting_counts == 'random':
            name = name + '_randomstart'        
        if starting_counts == 'subonly':
            name = name + '_subonlystart'
        if starting_counts == 'CHM13':
            name = name + '_CHM13start'
        if starting_counts == 'uniform':
            name = name + '_uniformstart'
        if stochastics is None:
            name = name
        else:
            name = name + '_stochastics_' + str(stochastics)
        return A_count_input, B_count_input, None, None, None, None, name

In [ ]:
def run_simulation_constantspeedup(exp_int, con_int, exp_power, con_power, min_speedup = 1, rounds = 9, ceiling = 0.1, interp = False, nonexp_factor = 0.01, A_bins = 200, B_bins = 200, input_nuc = 'A', mutonly = False, stochastics = None, boot = None, boundary_count = 1000, overwrite = False, starting_counts = 'random', reflective = True, sim_dir = 'simulations/testing/', rates_function = 'powerlaw', first_bin = 0, write = True, startfrom = None, recnum = 11, pin_len = 9):
    starting_conditions = setup_evolve(exp_power, con_power, exp_int, con_int, nonexp_factor = nonexp_factor, A_bins = A_bins, B_bins = B_bins, input_nuc = input_nuc, mutonly = mutonly, starting_counts = starting_counts, boot = boot, ceiling = None, interp = interp, rates_function = rates_function, pin_len = pin_len)

    # set speedup to maximum allowable or reduce A_bins to avoid rate ceiling
    if mutonly != True:
        rates_sum = starting_conditions[2][:A_bins] + starting_conditions[3][:A_bins] + starting_conditions[4][:A_bins]
    else:
        rates_sum = denovo_substitution_context_rate.loc[input_nuc].sum() * 100
    speedup = -2 - round(np.log10(rates_sum.max()) - 0.5)
    if speedup < min_speedup:
        speedup = min_speedup; A_bins = np.array(list(np.where(rates_sum * 10**speedup >= ceiling)[0])).min() -1
        starting_conditions = setup_evolve(exp_power, con_power, exp_int, con_int, nonexp_factor = nonexp_factor, A_bins = A_bins, B_bins = B_bins, input_nuc = input_nuc, mutonly = mutonly, starting_counts = starting_counts, boot = boot, ceiling = None, interp = interp, rates_function = rates_function, pin_len = pin_len)
    rounds = rounds - speedup # default is 10^9 generations = 10^(iterations+speedup)

    if overwrite == False:
        if 'Adist_'+input_nuc+'_bins'+str(A_bins)+'_sp1e'+str(speedup)+'_rounds1e'+str(rounds)+'_'+starting_conditions[6]+'.pickle' in os.listdir(sim_dir):
            print('already done: ' + starting_conditions[6])
            return None
        else:
            pass
    else:
        pass

    if startfrom is None:
        A_counts_timeseries = dict(); B_counts_timeseries = dict()
        A_counts_timeseries[0] = starting_conditions[0][:A_bins]; B_counts_timeseries[0] = starting_conditions[1]
        A_counts_current = A_counts_timeseries[0]; B_counts_current = B_counts_timeseries[0]; flag = False; boundary_flag = False
        startrep = 0
        print('\r' + '         ' + starting_conditions[6], end = '     ')

    else:
        A_counts_timeseries, B_counts_timeseries = startfrom
        startrep = list(A_counts_timeseries.keys())[-1]
        A_counts_current = pd.Series(A_counts_timeseries[startrep]).reindex(range(A_bins)).fillna(0).values; B_counts_current = B_counts_timeseries[startrep]; flag = False; boundary_flag = False
        
    for rep in range(1, 1 + 10**rounds):
        if (flag == False):
            A_counts_current, B_counts_current, flag, boundary_flag = mut_evolve_dist_AB(A_counts_current, B_counts_current, starting_conditions[2:6], boot=boot, input_nuc = input_nuc, mutonly=mutonly, speedup_multiplier=10**speedup, stochastics = stochastics, reflective = reflective, boundary_count = boundary_count, first_bin = first_bin)
            # save 10 evenly-spaced (in log scale) timepoints per log10 rounds
            if rep in np.linspace(0,10**rounds,recnum).astype(int):
                print('\r' + str(rep), end = '   ')
                A_counts_timeseries[startrep + rep * (10**speedup)], B_counts_timeseries[startrep + rep * (10**speedup)] = A_counts_current, B_counts_current
        else:
            print('\r' + 'ending due to numerical error at round '+str(rep))
            break
    if write == True:
        A_counts_timeseries = pd.DataFrame(A_counts_timeseries)
        B_counts_timeseries = pd.DataFrame(B_counts_timeseries)
        A_counts_timeseries.to_pickle(sim_dir + 'Adist_'+input_nuc+'_bins'+str(A_bins)+'_sp1e'+str(speedup)+'_rounds1e'+str(rounds)+'_'+starting_conditions[6]+'.pickle')
        B_counts_timeseries.to_pickle(sim_dir + 'Bdist_'+input_nuc+'_bins'+str(A_bins)+'_sp1e'+str(speedup)+'_rounds1e'+str(rounds)+'_'+starting_conditions[6]+'.pickle')
    return A_counts_timeseries, B_counts_timeseries

In [ ]:
def run_simulation_prospeedup(exp_int, con_int, exp_power, con_power, min_speedup = 3, interp = False, nonexp_factor = 0.01, A_bins = 200, B_bins = 200, input_nuc = 'A', mutonly = False, stochastics = None, boot = None, boundary_count = 1000, overwrite = False, starting_counts = 'subonly', reflective = True, sim_dir = 'simulations/testing/', rates_function = 'powerlaw', first_bin = 0, pin_len= 9):
    starting_conditions = setup_evolve(exp_power, con_power, exp_int, con_int, nonexp_factor = nonexp_factor, A_bins = A_bins, B_bins = B_bins, input_nuc = input_nuc, mutonly = mutonly, starting_counts = starting_counts, ceiling = None, interp = interp, boot = boot, rates_function = rates_function, pin_len = pin_len)
    if overwrite == False:
        if 'Adist_'+input_nuc+'_prospeedup_'+starting_conditions[6]+'.pickle' in os.listdir(sim_dir):
            print('already done: ' + starting_conditions[6])
            return None
        else:
            pass
    else:
        pass

    # set speedup to minimum, then increase to maximum allowable or reduce A_bins to avoid rate ceiling
    exp_rate = starting_conditions[2].copy(); con_rate = starting_conditions[3].copy()
    max_speedup=0
    exp_rate *= 10**max_speedup
    con_rate *= 10**max_speedup
    while exp_rate.max() + con_rate.max() < 0.01:
        max_speedup += 1
        exp_rate *= 10
        con_rate *= 10
    
    A_counts_timeseries, B_counts_timeseries = run_simulation_constantspeedup(exp_int=exp_int, con_int=con_int, exp_power=exp_power, con_power=con_power, min_speedup = min_speedup, rounds = 9, ceiling = 0.1, interp = interp, nonexp_factor = nonexp_factor, A_bins = A_bins, B_bins = B_bins, input_nuc = input_nuc, mutonly = mutonly, stochastics = stochastics, boot = boot, boundary_count = boundary_count, overwrite = None, starting_counts = starting_counts, reflective = reflective, sim_dir = None, rates_function = rates_function, first_bin = first_bin, write = False, recnum = 5, pin_len = pin_len)
    if max_speedup < min_speedup:
        for speedup in list(range(max_speedup, min_speedup))[::-1]:
            A_counts_timeseries, B_counts_timeseries = run_simulation_constantspeedup(exp_int=exp_int, con_int=con_int, exp_power=exp_power, con_power=con_power, min_speedup = speedup, rounds = 9 - min_speedup +speedup, ceiling = 0.1, interp = interp, nonexp_factor = nonexp_factor, A_bins = A_bins, B_bins = B_bins, input_nuc = input_nuc, mutonly = mutonly, stochastics = stochastics, boot = boot, boundary_count = boundary_count, overwrite = None, starting_counts = starting_counts, reflective = reflective, sim_dir = None, rates_function = rates_function, first_bin = first_bin, write = False, startfrom = (A_counts_timeseries, B_counts_timeseries), recnum = 5, pin_len = pin_len)
            
    A_counts_timeseries = pd.DataFrame.from_dict(A_counts_timeseries, orient = 'index').T
    B_counts_timeseries = pd.DataFrame.from_dict(B_counts_timeseries, orient = 'index').T
    A_counts_timeseries.to_pickle(sim_dir + 'Adist_'+input_nuc+'_prospeedup_'+starting_conditions[6]+'.pickle')
    B_counts_timeseries.to_pickle(sim_dir + 'Bdist_'+input_nuc+'_prospeedup_'+starting_conditions[6]+'.pickle')
    return A_counts_timeseries, B_counts_timeseries

#### input arguments and run

In [ ]:
# used for running script. ignore if running demo via Jupyter notebook

import argparse
parser = argparse.ArgumentParser(description='repeat distribution simulation')

parser.add_argument('--dir', action="store", dest='dir', default = 'simulations/grid_output/', type=str)
parser.add_argument('--exp_p', action="store", dest='exp_p', type=float)
parser.add_argument('--con_p', action="store", dest='con_p', type=float)
parser.add_argument('--exp_i', action="store", dest='exp_i', type=int)
parser.add_argument('--con_i', action="store", dest='con_i', type=int)
parser.add_argument('--motif', action="store", dest='motif', default = 'A', type=str)
parser.add_argument('--speedup', action="store", dest='speedup', default = 5, type=int)
parser.add_argument('--rounds', action="store", dest='rounds', default = 0, type=int)
parser.add_argument('--A_bins', action="store", dest='A_bins', default = 200, type=int)
parser.add_argument('--B_bins', action="store", dest='B_bins', default = 200, type=int)
parser.add_argument('--boot', action="store", dest='boot', type = int)
parser.add_argument('--boundary_count', action="store", dest='boundary_count', default = 1000, type=int)
parser.add_argument('--mutonly', default=False, action="store_true")
parser.add_argument('--starting_counts', action="store", dest='starting_counts', default = 'random', type=str)
parser.add_argument('--overwrite', default=False, action="store_true")
parser.add_argument('--stochastics', action="store", dest='stochastics', type = int)
parser.add_argument('--reflective', default=True, action="store_false")
parser.add_argument('--ceiling', action="store", dest='ceiling', default=None, type=float)
parser.add_argument('--constantspeedup', default=False, action="store_true")
parser.add_argument('--interp', default=False, action="store_true")
parser.add_argument('--jobgroup', action="store", dest='jobgroup', type=int)
parser.add_argument('--rates_function', action="store", dest='rates_function', default = 'powerlaw', type=str)
parser.add_argument('--jobfile', action="store", dest='jobfile', default = 'grid_group_jobs.pickle', type=str)
parser.add_argument('--firstbin', action="store", dest='firstbin', default = 0, type=int)
parser.add_argument('--pin_len', action="store", dest='pin_len', default = 9, type=int)
parser.add_argument('--recnum', action="store", dest='recnum', default = 11, type=int)


args = parser.parse_args()
finished = os.listdir(args.dir)

if args.constantspeedup == False:
    if args.jobgroup is not None:
        if args.jobgroup == -1:
            grid_group = pd.read_pickle(args.jobfile)
        else:
            grid_group = pd.read_pickle(args.jobfile).loc[args.jobgroup]
        if __name__ == '__main__':
            pool = multiprocessing.Pool()
            for exp_i, con_i, exp_p, con_p in grid_group:
                pool.apply_async(run_simulation_prospeedup, args=(exp_i, con_i, exp_p, con_p, args.speedup, args.interp, 0.01, args.A_bins, args.B_bins, args.motif, args.mutonly, args.stochastics, args.boot, args.boundary_count, args.overwrite, args.starting_counts, args.reflective, args.dir, args.rates_function, args.firstbin, args.pin_len))
            pool.close()
            pool.join()
    else:
        run_simulation_prospeedup(exp_power=args.exp_p, con_power=args.con_p, exp_int=args.exp_i, con_int=args.con_i, boot = args.boot, input_nuc = args.motif, A_bins = args.A_bins, B_bins = args.B_bins, mutonly = args.mutonly, starting_counts = args.starting_counts, min_speedup = args.speedup, sim_dir = args.dir, overwrite = args.overwrite, stochastics = args.stochastics, reflective = args.reflective, rates_function = args.rates_function, first_bin = args.firstbin)
else:
    if args.jobgroup is not None:
        if args.jobgroup == -1:
            grid_group = pd.read_pickle(args.jobfile)
        else:
            grid_group = pd.read_pickle(args.jobfile).loc[args.jobgroup]
        if __name__ == '__main__':
            pool = multiprocessing.Pool()
            for exp_i, con_i, exp_p, con_p in grid_group:
                pool.apply_async(run_simulation_constantspeedup, args=(exp_i, con_i, exp_p, con_p, args.speedup, args.rounds, args.ceiling, args.interp, 0.01, args.A_bins, args.B_bins, args.motif, args.mutonly, args.stochastics, args.boot, args.boundary_count, args.overwrite, args.starting_counts, args.reflective, args.dir, args.rates_function, args.firstbin, True, None, args.recnum, args.pin_len))
            pool.close()
            pool.join()
    else:
        run_simulation_constantspeedup(exp_power=args.exp_p, con_power=args.con_p, exp_int=args.exp_i, con_int=args.con_i, boot = args.boot, input_nuc = args.motif, A_bins = args.A_bins, B_bins = args.B_bins, mutonly = args.mutonly, starting_counts = args.starting_counts, min_speedup = args.speedup, rounds = args.rounds, ceiling = args.ceiling, sim_dir = args.dir, overwrite = args.overwrite, stochastics = args.stochastics, reflective = args.reflective, rates_function = args.rates_function, first_bin = args.firstbin)

##### - remove all lines below to generate simulation_script_prospeedup.py
- Generation of complete dataset requires using simulation_script_speedup.py along with jobs_prospeedup.sh (and simulation_script.py along with jobs_sp1_long.sh and jobs_sp2_long.sh) on a Slurm-based cluster to more quickly populate all points in the grid.

##### Arguments:
- --dir -> directory where output of simulation will be stored
- --exp_p -> power law exponent controlling expansion and non-motif insertion rates
- --con_p -> power law exponent controlling contraction rate
- --exp_i -> intercept of expansion rate power law (as the index of the 'intercept_list')
- --con_i -> intercept of contraction rate power law (as the index of the 'intercept_list')
- --motif -> STR sequence motif
- --speedup -> controls constant or maximum speedup factor applied to all mutation rates
- --rounds -> controlls number of iterations (10^n). with prospeedup==True, this adds rounds per speedup
- --A_bins -> number of length bins 1-n for the A (repeat) motifs
- --B_bins -> number of length bins 1-n for the B (non-repeat) strings
- --boot -> selects from a pre-generated list of mutation rates subject to Poisson noise
- --mutonly -> turns off all indel and instability processes in the simulation, running substituion processes only
- --starting_counts -> choose the starting distribution. recognizes 'random', 'subonly', 'CHM13' and 'uniform', or supply (A_counts, B_counts)
- --overwrite -> ignore check for whether the conditions will produce a file that already exists in the given directory
- --stochastics -> turns on stochastic (Poisson) sampling of mutation counts during each iteration
- -- ceiling -> set upper rate limit for any mutation process
- --reflective -> turn off handling of the reflecting boundary at L=A_bins and B_bins, turning it into an absorbing boundary
- --rates_function -> choose between several functions determining the expansion and contraction rates
- --jobgroup -> run a list of jobs in parallel, according to a pre-made file specifying groups of parameter combinations
- --jobfile -> location of pre-made file specifying groups of parameter combinations
- --firstbin -> flag to ignore first n bins (testing purposes only)
- --pin_len -> length bin at which to pin the power law model to the empirical rates (default is 9)
- --recnum -> number of linearly-spaced timepoints to record

## Make job files for running grid on cluster

### for power law 4 parameter model:
- split into 1000 jobs by approximate time needed

In [ ]:
def find_constantspeedup(exp_int, con_int, exp_power, con_power, min_speedup = 0, rounds = 9, ceiling = 0.1, interp = False, nonexp_factor = 0.01, A_bins = 200, B_bins = 200, input_nuc = 'A', mutonly = False, stochastics = None, boot = None, boundary_count = 1000, overwrite = False, starting_counts = 'random', reflective = True, sim_dir = 'simulations/testing/', rates_function = 'powerlaw', first_bin = 0, write = True, startfrom = None, recnum = 10):
    starting_conditions = setup_evolve(exp_power, con_power, exp_int, con_int, nonexp_factor = nonexp_factor, A_bins = A_bins, B_bins = B_bins, input_nuc = input_nuc, mutonly = mutonly, starting_counts = starting_counts, boot = boot, ceiling = None, interp = interp, rates_function = rates_function)

    # set speedup to maximum allowable or reduce A_bins to avoid rate ceiling
    rates_sum = starting_conditions[2][:A_bins] + starting_conditions[3][:A_bins] + starting_conditions[4][:A_bins]
    speedup = -2 - round(np.log10(rates_sum.max()) - 0.5)
    if speedup < min_speedup:
        speedup = min_speedup; A_bins = np.array(list(np.where(rates_sum * 10**speedup >= ceiling)[0])).min() -1
    return speedup

In [ ]:
jobs_fullgrid = pd.Series([(exp_c, con_c, exp_power, con_power) for exp_power in np.linspace(0,4,41).round(2) for exp_c in range(8) for con_power in np.linspace(0,4,41).round(2) for con_c in range(8)]).reset_index().set_index(['index'])
jobs_fullgrid['speedup'] = [find_constantspeedup(exp_c, con_c, exp_power, con_power) for exp_power in np.linspace(0,4,41).round(2) for exp_c in range(8) for con_power in np.linspace(0,4,41).round(2) for con_c in range(8)]

In [ ]:
jobs_per_speedup = round(((jobs_fullgrid['speedup'].value_counts().sort_index(ascending = False) * np.array([0.2, 0.4, 0.6, 0.8, 1])) / (jobs_fullgrid['speedup'].value_counts().sort_index(ascending = False) * np.array([0.2, 0.4, 0.6, 0.8, 1])).sum() * 1000) + 0.5).astype(int)
jobs_fullgrid_split = dict()
for speedup in jobs_per_speedup.index:
    jobs_fullgrid_split[speedup] = pd.DataFrame(np.array_split(jobs_fullgrid.loc[jobs_fullgrid['speedup'] == speedup][0].values, jobs_per_speedup[speedup])).T.unstack()
for speedup in [3,2,1,0]:
    jobs_fullgrid_split[speedup].index = pd.MultiIndex.from_arrays([jobs_fullgrid_split[speedup].index.get_level_values(0) + jobs_per_speedup.cumsum()[speedup+1], jobs_fullgrid_split[speedup].index.get_level_values(1)])
jobs_fullgrid_split = pd.concat(jobs_fullgrid_split).dropna()
jobs_fullgrid_split.index = jobs_fullgrid_split.index.droplevel(0)

In [ ]:
jobs_fullgrid_split

In [ ]:
jobs_fullgrid_split.to_pickle('simulations/fullgrid_jobs_prospeedup.pickle')

In [ ]:
jobs_fullgrid_split = pd.read_pickle('simulations/fullgrid_jobs_prospeedup.pickle')

### sparse grid 

#### for progressive speedup
- all jobs together

In [ ]:
jobs_3Mgrid = pd.Series([(exp_c, exp_c-1, exp_power, con_power) for exp_power in np.linspace(0,4,41).round(2) for con_power in np.linspace(0,4,41).round(2) for exp_c in range(1,8)]).reset_index().set_index(['index'])

In [ ]:
jobs_3Mgrid['speedup'] = [find_constantspeedup(exp_c, exp_c-1, exp_power, con_power) for exp_power in np.linspace(0,4,41).round(2) for con_power in np.linspace(0,4,41).round(2) for exp_c in range(1,8)]

In [ ]:
jobs_per_speedup = round(((jobs_3Mgrid['speedup'].value_counts().sort_index(ascending = False) * np.array([0.2, 0.4, 0.6, 0.8, 1])) / (jobs_3Mgrid['speedup'].value_counts().sort_index(ascending = False) * np.array([0.2, 0.4, 0.6, 0.8, 1])).sum() * 198) + 0.5).astype(int)
jobs_3Mgrid_split = dict()
for speedup in jobs_per_speedup.index:
    jobs_3Mgrid_split[speedup] = pd.DataFrame(np.array_split(jobs_3Mgrid.loc[jobs_3Mgrid['speedup'] == speedup][0].values, jobs_per_speedup[speedup])).T.unstack()
for speedup in [3,2,1,0]:
    jobs_3Mgrid_split[speedup].index = pd.MultiIndex.from_arrays([jobs_3Mgrid_split[speedup].index.get_level_values(0) + jobs_per_speedup.cumsum()[speedup+1], jobs_3Mgrid_split[speedup].index.get_level_values(1)])
jobs_3Mgrid_split = pd.concat(jobs_3Mgrid_split).dropna()
jobs_3Mgrid_split.index = jobs_3Mgrid_split.index.droplevel(0)

In [ ]:
jobs_3Mgrid_split.to_pickle('simulations/3Mgrid_jobs_prospeedup.pickle')

#### for constant speedup
- separate into shorter jobs, and jobs expected to take > 12 hours

In [ ]:
jobs_3Mgrid = pd.Series([(exp_c, exp_c-1, exp_power, con_power) for exp_power in np.linspace(0,4,21).round(2) for con_power in np.linspace(0,4,21).round(2) for exp_c in range(1,8)]).reset_index().set_index(['index'])

In [ ]:
jobs_3Mgrid['speedup'] = [find_constantspeedup(exp_c, exp_c-1, exp_power, con_power) for exp_power in np.linspace(0,4,21).round(2) for con_power in np.linspace(0,4,21).round(2) for exp_c in range(1,8)]

In [ ]:
slow_jobs = pd.DataFrame(np.array_split(jobs_3Mgrid.loc[jobs_3Mgrid['speedup'] == 0][0].values, 20)).unstack().dropna()

fast_jobs = dict()
fast_jobs[4] = pd.DataFrame(np.array_split(jobs_3Mgrid.loc[jobs_3Mgrid['speedup'] == 4][0].values, 2000)).unstack().dropna()
fast_jobs[3] = pd.DataFrame(np.array_split(jobs_3Mgrid.loc[jobs_3Mgrid['speedup'] == 3][0].values, 200)).unstack().dropna()
fast_jobs[2] = pd.DataFrame(np.array_split(jobs_3Mgrid.loc[jobs_3Mgrid['speedup'] == 2][0].values, 20)).unstack().dropna()
fast_jobs[1] = pd.DataFrame(np.array_split(jobs_3Mgrid.loc[jobs_3Mgrid['speedup'] == 1][0].values, 20)).unstack().dropna()

fast_jobs[3].index = pd.MultiIndex.from_arrays([fast_jobs[3].index.get_level_values(0) + len(fast_jobs[4].index.levels[0]), fast_jobs[3].index.get_level_values(1)])
fast_jobs[2].index = pd.MultiIndex.from_arrays([fast_jobs[2].index.get_level_values(0) + len(fast_jobs[4].index.levels[0]) + len(fast_jobs[3].index.levels[0]), fast_jobs[2].index.get_level_values(1)])
fast_jobs[1].index = pd.MultiIndex.from_arrays([fast_jobs[1].index.get_level_values(0) + len(fast_jobs[4].index.levels[0]) + len(fast_jobs[3].index.levels[0]) + len(fast_jobs[2].index.levels[0]), fast_jobs[1].index.get_level_values(1)])

fast_jobs = pd.concat(fast_jobs)
fast_jobs.index = fast_jobs.index.droplevel(0)

In [ ]:
len(slow_jobs.index.levels[0]), len(fast_jobs.index.levels[0])

In [ ]:
slow_jobs.to_pickle('simulations/3Mgrid_jobs_constantspeedup_medium.pickle')
fast_jobs.to_pickle('simulations/3Mgrid_jobs_constantspeedup_short.pickle')

### jobs for power law only model

In [ ]:
plonly_lambda_list = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
spacing = np.linspace(0,4,21).round(2)
jobs_xaxis = pd.Series([(exp_L, con_L, exp_power, con_power) for exp_power in spacing for exp_L in plonly_lambda_list for con_power in spacing for con_L in plonly_lambda_list])
jobs_xaxis = jobs_xaxis.sample(len(jobs_xaxis), replace=False)     # random order
jobs_xaxis_group = dict()
for n in range(1000):
    jobs_xaxis_group[n] = jobs_xaxis[n::1000]
jobs_xaxis_group = pd.concat(jobs_xaxis_group)

In [ ]:
jobs_xaxis_group.to_pickle('simulations/grid_group_jobs_xaxis_v2.pickle')

### jobs for log rate curves (full grid, random order)

In [ ]:
jobs_log = pd.Series([(exp_L, con_L, exp_power, con_power) for exp_power in np.linspace(0,4,41).round(2) for exp_L in range(8) for con_power in np.linspace(0,4,41).round(2) for con_L in range(8)])
jobs_log = jobs_log.sample(len(jobs_log), replace=False)    #random order
jobs_log_group = dict()
for n in range(1000):
    jobs_log_group[n] = jobs_log[n::1000]
jobs_log_group = pd.concat(jobs_log_group)

In [ ]:
jobs_log_group.to_pickle('simulations/grid_group_jobs_log.pickle')

## Substitution-only simulations
- note: run this set of computational models first, and process data to generate the "subonly_counts.pickle" file which is used to initiate runs of the computational model for all other parameter combinaztions

In [ ]:
# directory to store simulation results (create if it does not exist, can be changed as needed)
sim_dir = 'simulations/subonly_output/'
finished = os.listdir(sim_dir)

In [ ]:
mutonly_jobs = [(0,0,0,0,3,10, 0.1, False, 0.01, 100, 200, 'A', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 200, 'C', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AC', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AG', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AT', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'CG', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AAC', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AAG', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AAT', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'ACC', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'ACG', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'ACT', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AGC', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'AGG', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'ATC', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9),
(0,0,0,0,3,10, 0.1, False, 0.01, 100, 500, 'CCG', True, None, None, 1000, False, 'random', True, 'simulations/subonly_output/', 'powerlaw', 0, True, None, 11, 9)]

In [ ]:
if __name__ == '__main__':
    pool = multiprocessing.Pool()
    for args in mutonly_jobs:
        pool.apply_async(run_simulation_constantspeedup, args=args)
    pool.close()
    pool.join()

#### Process data

In [ ]:
grid_files_mutonly = os.listdir('simulations/subonly_output/')
grid_files_A_mutonly = [file for file in grid_files_mutonly if file.startswith('Adist')]
grid_files_B_mutonly = [file for file in grid_files_mutonly if file.startswith('Bdist')]

In [ ]:
substitutions_only = dict()
for file in grid_files_A_mutonly:
    motif = file.split('_')[1]
    substitutions_only[motif] = pd.read_pickle('simulations/subonly_output/' + file)

In [ ]:
substitutions_only_B = dict()
for file in grid_files_B_mutonly:
    motif = file.split('_')[1]
    substitutions_only_B[motif] = pd.read_pickle('simulations/subonly_output/' + file)

In [ ]:
starting_subonlydist = dict()
starting_subonlydist['A'] = dict(); starting_subonlydist['B'] = dict()
for motif in substitutions_only:
    starting_subonlydist['A'][motif] = substitutions_only[motif].T.iloc[-1]
    starting_subonlydist['B'][motif] = substitutions_only_B[motif].T.iloc[-1]
starting_subonlydist['A'] = pd.concat(starting_subonlydist['A'], axis=1)
starting_subonlydist['B'] = pd.concat(starting_subonlydist['B'], axis=1)
starting_subonlydist = pd.concat(starting_subonlydist, axis=1).sort_index(axis=1)
starting_subonlydist.index +=1

In [ ]:
starting_subonlydist.to_pickle('repeat_distributions/subonly_counts_remake.pickle')

## Run demo
- For a demo of the computational model, the below code can be run, requiring values for "mult", "exp_power" and "con_power" parameters. See Methods for further description, including default values for optional paramerters.
- Demo on a "normal" desktop computer with given parameters is expected to take ~10 minutes.

In [ ]:
# For demo purposes only:
# If data files (above) have not been created, it is possible to substitute these example data and run a demo of the computational model:
CHM13_counts = dict()
CHM13_counts['A'] = pd.DataFrame([861201679, 234254484, 88812020, 32801187, 12627047, 3780464, 1581788, 567751, 326510, 207206, 127161, 97987, 86634, 77081, 68340, 58903, 47652, 38435, 32238, 27356, 24095, 21886, 19548, 16572, 13746, 10757, 8409, 6853, 5435, 3908, 2740, 1824, 1408, 952, 912, 786, 809, 803, 693, 636, 497, 341, 241, 200, 163, 157, 167, 119, 116, 85, 94, 60, 42, 63, 26, 29, 20, 14, 7, 3, 3, 7, 4, 4], index = range(1,65), columns = ['A'])
CHM13_counts['B'] = pd.DataFrame([391005710, 234634738, 173991271, 128644266, 82379768, 57163482, 44845809, 31123715, 23724537, 16138191, 12638350, 9637450, 6485630, 4960219, 3700614, 2978004, 2541592, 2199129, 1529461, 1307551, 948882, 703894, 583040, 484866, 436960, 322284, 281835, 207162, 183167, 146508, 125876, 104629, 89550, 80680, 65894, 60237, 50645, 43932, 38951, 33281, 29366, 26809, 23398, 20929, 18711, 16401, 15401, 13201, 11954, 11180, 9209, 8217, 7411, 7940, 6446, 6543, 5351, 5363, 4614, 4920, 3642, 3733, 3536, 2936, 2545, 2346, 1576, 1148, 793, 1602, 833, 2980, 2732, 2347, 2175, 1894, 1721, 1663, 1550, 1846, 1274, 1270, 1200, 1184, 863, 883, 726, 971, 667, 771, 1036, 924, 545, 982, 568, 536, 461, 578, 506, 660, 637, 453, 394, 553, 599, 371, 515, 433, 343, 501, 390, 526, 359, 333, 285, 317, 372, 327, 302, 539, 560, 518, 311, 277, 231, 297, 267, 588, 367, 361, 215, 230, 216, 402, 205, 511, 331, 215, 183, 184, 164, 340, 254, 344, 158, 401, 136, 170, 162, 187, 142, 175, 270, 235, 162, 366, 142, 351, 128, 127, 122, 111, 128, 117, 116, 136, 124, 342, 94, 112, 110, 98, 99, 134, 103, 128, 100, 149, 135, 158, 92, 83, 82, 82, 131, 191, 85, 309, 172, 396, 68, 70, 139, 278, 83, 100, 76, 149, 92, 282], index = range(1,201), columns = ['A'])
CHM13_counts = pd.concat(CHM13_counts, axis=1)

subonly_counts = dict()
subonly_counts['A'] = pd.DataFrame([916707580.2033024, 340290716.258333, 128678308.62519115, 48838020.7874859, 18548753.855557196, 7045762.038399432, 2676403.5444367407, 1016663.2352091095, 386191.7452785688, 146699.59581817046, 55725.61361521049, 21168.04760300643, 8040.938639205142, 3054.447695047391, 1160.2688617171016, 440.7421458419924, 167.42122927793446, 63.59697677514248, 24.15808002594318, 9.176738583082912, 3.4858950269147133, 1.3241593436114223, 0.5029979255644212, 0.19106983939869215, 0.07258018705917339, 0.027570460990195812, 0.010472972721773027, 0.003978285225988993, 0.0015111997099370791, 0.0005740474685914903, 0.00021805886675958894, 8.283229519180028e-05, 3.146484813344478e-05, 1.1952302731299332e-05, 4.540226603820093e-06, 1.7246599318518845e-06, 6.551329129767819e-07, 2.4886015250820107e-07, 9.453253573385976e-08, 3.5909325869190544e-08, 1.3640591298746398e-08, 5.18154341457788e-09, 1.968271871001872e-09, 7.476718514560248e-10, 2.840121863729806e-10, 1.0788546051489958e-10, 4.098159568134373e-11, 1.55673542715905e-11, 5.9134476095456306e-12, 2.246294522548187e-12, 8.532821147996443e-13, 3.2412952091919334e-13, 1.2312451475204588e-13, 4.677033455618555e-14, 1.776627667449281e-14, 6.74873485233351e-15, 2.563588473914226e-15, 9.738100558675044e-16, 3.69913515588853e-16, 1.405161183033852e-16, 5.3376745295775186e-17, 2.0275801614578305e-17, 7.702008221664194e-18, 2.925700880991615e-18, 1.1113628288474581e-18, 4.2216459835948847e-19, 1.6036432340719404e-19, 6.091632581647316e-20, 2.3139802370857326e-20, 8.789933512660019e-21, 3.3389624474187368e-21, 1.2683452280060184e-21, 4.817962593886852e-22, 1.830161303368957e-22, 6.952088836470106e-23, 2.6408349417727886e-23, 1.0031530599987096e-23, 3.810598102390866e-24, 1.4475017299912813e-24, 5.49851021290328e-25, 2.0886755390297206e-25, 7.93408639505333e-26, 3.013858579147787e-26, 1.1448505955972263e-26, 4.348853311273552e-27, 1.6519644740485116e-27, 6.275186646154749e-28, 2.383705462510538e-28, 9.054793169731035e-29, 3.439572553323957e-29, 1.306563158508199e-29, 4.963137692955658e-30, 1.8853075015279892e-30, 7.161565258914736e-31, 2.7204041120994567e-31, 1.0333754823401774e-31, 3.9253740392858455e-32, 1.4911021301878577e-32, 5.682823327959757e-33, 3.428759707012527e-33], index = range(1,101), columns = ['A'])
subonly_counts['B'] = pd.DataFrame([550498794.3648516, 346320080.1269246, 215422559.61719304, 133909901.19805124, 83238781.58012784, 51741505.78915495, 32162697.296232127, 19992442.949264973, 12427371.10493236, 7724896.5005180165, 4801822.158505596, 2984829.122353852, 1855380.0194100519, 1153310.58339157, 716901.8141016357, 445628.6263764576, 277004.282511156, 172186.81203998503, 107031.91290661399, 66531.40414603188, 41356.148997399345, 25707.124054394466, 15979.636478957083, 9932.99684007114, 6174.384902609636, 3838.018831515179, 2385.725668777034, 1482.9752579443784, 921.8225064420567, 573.0080315439691, 356.1837576315802, 221.4050453336747, 137.6261355238746, 85.54887785276219, 53.177475877009144, 33.05530138591516, 20.547288709994625, 12.772265132387137, 7.939283810843722, 4.935086045880151, 3.0676664117959866, 1.9068719626311865, 1.185318151897783, 0.7367978284602383, 0.45799605713836933, 0.28469191988886655, 0.17696547379996758, 0.11000234544581367, 0.06837783519997773, 0.0425038968731665, 0.02642056807035858, 0.01642311572615223, 0.01020866506111151, 0.006345741214257145, 0.003944534502529505, 0.0024519361751922765, 0.0015241319358117495, 0.0009474056385579074, 0.0005889105942086762, 0.0003660688451243766, 0.0002275496496217057, 0.00014144564262323002, 8.79230965653398e-05, 5.4653298371514975e-05, 3.3972677710072244e-05, 2.111753297938136e-05, 1.3126730925983014e-05, 8.159620963840785e-06, 5.072048375865068e-06, 3.152802665849103e-06, 1.9597929501391797e-06, 1.218214019233847e-06, 7.572460124180416e-07, 4.707067184168981e-07, 2.9259291053287115e-07, 1.8187675668200792e-07, 1.1305521572933016e-07, 7.027550983852459e-08, 4.3683497936869776e-08, 2.7153812137189442e-08, 1.6878902752873576e-08, 1.0491983840116927e-08, 6.521853138974554e-09, 4.054006278938213e-09, 2.5199842068591176e-09, 1.5664308256776053e-09, 9.736987735694173e-10, 6.052544971083951e-10, 3.762282712209035e-10, 2.338647837267046e-10, 1.453711516416722e-10, 9.036320643437254e-11, 5.617007903485778e-11, 3.4915513772451004e-11, 2.1703603109364986e-11, 1.3491034128803281e-11, 8.386073084150692e-12, 5.212811790503936e-12, 3.2403016871595538e-12, 2.014182641877817e-12, 1.2520228381568401e-12, 7.782616902133947e-13, 4.837701358111617e-13, 3.007131755882505e-13, 1.8692434129019585e-13, 1.161928116332889e-13, 7.222585021331858e-14, 4.489583620284987e-14, 2.79074057612331e-14, 1.734733913415051e-14, 1.0783165501297115e-14, 6.702852658218828e-15, 4.166516200867617e-15, 2.58992076020143e-15, 1.6099036270939265e-15, 1.000721616026816e-15, 6.220519886591289e-16, 3.8666964957855806e-16, 2.403551803242204e-16, 1.4940560442655428e-16, 9.287103612226439e-17, 5.772895456985953e-17, 3.5884516151424185e-17, 2.230593831148531e-17, 1.3865447756247434e-17, 8.61880987908215e-18, 5.357481780441848e-18, 3.3302290490741228e-18, 2.070081798464368e-18, 1.2867699456062506e-18, 7.998606113747827e-19, 4.971960992820773e-19, 3.0905880052828883e-19, 1.9211201037559625e-19, 1.1941748452872465e-19, 7.423031794465943e-20, 4.6141820219298135e-20, 2.8681913699161395e-20, 1.782877592466713e-20, 1.1082428261447667e-20, 6.88887541629852e-21, 4.282148585285124e-21, 2.661798246935351e-21, 1.6545829193632123e-21, 1.0284944173362715e-21, 6.393156572044061e-22, 3.9740080515485094e-22, 2.4702570343467985e-22, 1.5355202446965533e-22, 9.54484650418752e-23, 5.933109322599955e-23, 3.6880411034874887e-23, 2.292498998662099e-23, 1.425025240064973e-23, 8.858005765792488e-24, 5.506166763982025e-24, 3.4226521447819294e-24, 2.1275323117363145e-24, 1.322481381691934e-24, 8.2205896252379515e-25, 5.1099467048914545e-25, 3.1763603971503663e-25, 1.9744365167110993e-25, 1.2273165104374322e-25, 7.629041521686596e-26, 4.742238374913899e-26, 2.9477916381210484e-26, 1.832357392184888e-26, 1.1389996393485771e-26, 7.080060822028102e-27, 4.400990089187962e-27, 2.7356705333475412e-27, 1.700502185953998e-27, 1.057037990936665e-27, 6.570584404492206e-28, 4.0842978007157435e-28, 2.5388135206857206e-28, 1.5781351917303963e-28, 9.809742476498505e-29, 6.097769567492104e-29, 3.7903944764414215e-29, 2.3561222063277495e-29, 1.4645736441560355e-29, 9.10384000199909e-30, 5.658978168336365e-30, 3.517640237820048e-30, 2.186577236145866e-30, 1.3591839092090395e-30, 8.448733795056932e-31, 5.251761903308292e-31, 3.2645132108644712e-31, 2.0292326080501292e-31, 1.2613779487458502e-31, 7.840768590403876e-32, 4.8738486469808327e-32, 3.0296010346166e-32, 1.8832103936251097e-32, 1.1706100394524177e-32, 7.276552153203449e-33, 4.52312977454541e-33], index = range(1,201), columns = ['A'])
subonly_counts = pd.concat(subonly_counts, axis=1)

denovo_exp_rate = pd.DataFrame([4.562984199878992e-12,  1.0467617654842797e-10,  5.807374246629577e-11,  8.005268166171157e-11,  1.3679705360645337e-10,  3.0556157928111613e-10,  2.289030948565612e-09,  4.854045051592342e-09,  1.1364615491677555e-08,  1.419922964171028e-08,  1.1828716797973438e-08,  4.1291073073549976e-10,  1.447422661312361e-10,  5.0615435868574976e-11, 1.6412002445322716e-10],  index = [0.0,  1.0,  2.0,  3.0,  4.0,  5.0,  6.0,  7.0,  8.0,  9.0,  10.0,  11.0,  12.0,  13.0,  25.0], columns = ['A'])
denovo_con_rate = pd.DataFrame([2.885531044935673e-10,  1.871202448033845e-10,  1.7746255255979686e-10,  2.498754225278858e-10,  3.7299996616692954e-10,  4.540853190555286e-10,  1.0654492863651945e-09,  3.20580338022747e-09,  7.138318574137602e-09,  1.3303065078067975e-08,  2.0285696564748838e-08,  3.1794126266633476e-09,  7.237113306561804e-10,  3.036926152114499e-10,  5.25055043000813e-11,  1.1059236545578058e-10,  1.3900672767207106e-10], [0.0,  1.0,  2.0,  3.0,  4.0,  5.0,  6.0,  7.0,  8.0,  9.0,  10.0,  11.0,  12.0,  13.0,  14.0,  20.0,  24.0], columns = ['A'])
denovo_nonexp_rate = pd.DataFrame([1.4427655224678364e-10,  5.807374246629577e-12,  8.791913014870057e-12,  6.383862501634492e-12,  1.0406121895022531e-11,  3.3700342391113206e-11,  2.1336461765241066e-11,  1.1596546420079138e-10,  1.3940337656658055e-10,  2.763718878031177e-10,  2.890375115148498e-10,  3.377319543062175e-10,  3.036926152114499e-10,  2.625275215004065e-10,  1.6588854818367085e-10,  6.036395505019818e-11,  6.996035254753307e-11,  8.281119434384756e-11,  9.221806429413932e-11, 1.1676038583818848e-10, 1.6412002445322716e-10, 1.0262244516086404e-09], index = [0.0,  2.0,  3.0,  4.0,  5.0,  6.0,  7.0,  8.0,  9.0,  10.0,  11.0,  12.0,  13.0,  14.0,  15.0,  16.0,  17.0,  18.0,  19.0, 22.0,  25.0, 32.0], columns = ['A'])
denovo_substitution_context_rate = pd.DataFrame([4.541573240083719e-09,  8.077760235763368e-09,  6.447333857760922e-09,  2.865765641003364e-09,  4.7677112600042175e-09,  3.967453912936841e-09], index = ['Afission', 'Acontraction', 'A10', 'Afusion', 'Aexpansion', 'A01'], columns = ['A']).transpose()

intercept_list = dict()
intercept_list['A'] = [denovo_con_rate['A'][8]] + [denovo_exp_rate['A'][8] * (denovo_exp_rate['A'][8]/ denovo_con_rate['A'][8])**x for x in range(7)]

In [ ]:
# directory to store simulation results (create if it does not exist, can be changed as needed)
sim_dir = 'simulations/testing/'
finished = os.listdir(sim_dir)
test = dict()

#### with constant speedup

In [ ]:
test[0] = run_simulation_constantspeedup(3,2,0.5,1, ceiling = 0.1, min_speedup = 4, input_nuc = 'A', mutonly = False, rounds = 9, interp = True, A_bins = 200, B_bins = 200, starting_counts = 'subonly', overwrite=True, sim_dir=sim_dir, first_bin=1, recnum = 101)

In [ ]:
# demo of multiprocessing
grid_group = pd.Series([(0, 0, 0.0, 0.0), (0, 1, 0.0, 0.0), (0, 2, 0.0, 0.0), (0, 3, 0.0, 0.0)])
if __name__ == '__main__':
    pool = multiprocessing.Pool()
    for exp_i, con_i, exp_p, con_p in grid_group:
        pool.apply_async(run_simulation_constantspeedup, args=(exp_i, con_i, exp_p, con_p, 2, 5, 0.1))
    pool.close()
    pool.join()


#### with progressive speedup

In [ ]:
test[1] = run_simulation_prospeedup(3,2,0.5,1, min_speedup=4, starting_counts = 'subonly', sim_dir=sim_dir, overwrite = True, interp = False, first_bin = 0)

In [ ]:
# demo of multiprocessing
grid_group = pd.Series([(0, 0, 0.0, 0.0), (0, 1, 0.0, 0.0), (0, 2, 0.0, 0.0), (0, 3, 0.0, 0.0)])
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=8)
    for exp_i, con_i, exp_p, con_p in grid_group:
        pool.apply_async(run_simulation_prospeedup, args=(exp_i, con_i, exp_p, con_p, 5, True))
    pool.close()
    pool.join()

#### Demo plots

In [ ]:
bootstrap_counts = pd.read_pickle('repeat_distributions/bootstrap_counts_1000.pickle').sort_index()
bootstrap_counts_max = dict(); bootstrap_counts_min = dict(); bootstrap_counts_mean = dict()
for motif in bootstrap_counts.columns.levels[2]:
    bootstrap_counts_mean[motif] = bootstrap_counts.xs(len(motif), level=1, drop_level=True, axis=1).xs(motif, level=1, drop_level=True, axis=1).apply(lambda x: x.sort_values().head(975).tail(950).mean(), axis=1).sort_index().replace(0, np.nan).dropna()
    bootstrap_counts_max[motif] = bootstrap_counts.xs(len(motif), level=1, drop_level=True, axis=1).xs(motif, level=1, drop_level=True, axis=1).apply(lambda x: x.sort_values().head(975).tail(950).max(), axis=1).reindex(bootstrap_counts_mean[motif].index)
    bootstrap_counts_min[motif] = bootstrap_counts.xs(len(motif), level=1, drop_level=True, axis=1).xs(motif, level=1, drop_level=True, axis=1).apply(lambda x: x.sort_values().head(975).tail(950).min(), axis=1).reindex(bootstrap_counts_mean[motif].index)

import plotly     # https://plotly.com/python/getting-started/
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "none"
pd.set_option('display.precision', 3)

def make_colorscale(list_of_traces, opacity = 0.15):
    current_colors = pd.Series(['rgb('+str(current/len(list_of_traces)*255) + ', 180, '+str((len(list_of_traces)-current)/len(list_of_traces)*255)+')' for current in range(len(list_of_traces))], index = list_of_traces)
    return pd.Series(current_colors, index = list_of_traces), pd.Series(['rgba' + color[3:-1] + ', '+str(opacity)+')' for color in current_colors], index = list_of_traces)
def loop_list(input_list, count):
    return input_list * (count // len(input_list)) + input_list[:(count % len(input_list))]
    
def make_evolve_fig(input_df, motif = 'A'):
    fig = go.Figure()
    colors = make_colorscale([rep for rep in input_df.columns] , opacity= 0.3)
    reps = input_df.columns[1:]
    
    fig.add_trace(go.Scattergl(x = input_df.index + 1, y = input_df[0] / input_df[0].sum(), name = 'start', legendgroup = 'start', connectgaps = True, line = dict(color = 'gray', dash = 'dot')))
    for rep in reps:
        fig.add_trace(go.Scattergl(x = input_df.index + 1, y = input_df[rep] / input_df[rep].sum(), name = '{:0.1e}'.format(rep), mode = 'lines', line = dict(color = colors[1][rep] if rep != reps[-1] else colors[0][rep])))
    fig.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index, y = bootstrap_counts_mean[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0.8)'), legendgroup = 'ci', name = 'T2T-CHM13'))
    fig.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index, y = bootstrap_counts_max[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
    fig.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index, y = bootstrap_counts_min[motif] / bootstrap_counts_mean[motif].sum(), fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))

    fig.update_xaxes(type = 'log', title = 'repeat length (units)', range = [0,2])
    fig.update_yaxes(type = 'log', title = 'repeat counts (normalized)', range = [-9.05,0.05], tickformat = '0.1e', dtick = 2)
        
    fig.update_layout(margin={'t':20,'l':60,'b':40,'r':10})        
    return fig

##### compare constant and progressive speedup

In [ ]:
make_evolve_fig(test[0][0])

In [ ]:
make_evolve_fig(test[1][0])

## Plot comparison of initial states (Fig. S13b)

In [ ]:
import plotly     # https://plotly.com/python/getting-started/
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "none"

In [ ]:
# Fig. S13b (left)
fig = go.Figure()
fig.add_trace(go.Scatter(x = subonly_counts['A']['A'].index, y = subonly_counts['A']['A'], line = dict(width = 3, color = 'black'), name = 'geometric<br>initial state'))
fig.add_trace(go.Scatter(x = counts_uniform['A']['A'].index, y = counts_uniform['A']['A'], line = dict(dash = 'dash', width = 4, color = 'red'), name = 'compound<br>initial state'))
fig.update_xaxes(type = 'log', range = [-0.1,1.9], title = 'repeat tract length (units)', tickvals = [1,5,10,20,70])
fig.update_yaxes(type = 'log', title = 'counts', range = [0,9.1], tickformat = '1.0e', )
fig.update_layout(width = 400, height = 400, legend = dict(xref = 'paper', x = 0.54))
fig.update_layout(font=dict(family = 'Arial', size = 16), margin={'t':50,'l':70,'b':45,'r':10})
fig.show()

In [ ]:
fig.write_image('plots/initial_state.pdf')